In [ ]:
%load_ext autoreload
%autoreload 2

import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

try:
    import text_recognizer
except ImportError:
    import sys
    sys.path.append('..')

from text_recognizer.datasets import IamDatasetW
from text_recognizer.datasets import IamWordsDataset
from text_recognizer.models import LineDetectorModel
from text_recognizer import util
from text_recognizer.networks import fcn_gated3_relu

In [ ]:
network = fcn_gated3_relu
dataset_args={"image_dim": 512, "num_classes": 3}

In [ ]:
paragraphs_dataset = IamWordsDataset(**dataset_args)
paragraphs_dataset.load_or_generate_data()
print(paragraphs_dataset)

In [ ]:
line_detector_model = LineDetectorModel(IamWordsDataset, network, dataset_args)
line_detector_model.load_weights()

In [ ]:
line_detector_model.name

In [ ]:
# Let's look at some test images, ground truth, and predictions

for ind in range(5):
    image = paragraphs_dataset.x_test[ind]
    gt = paragraphs_dataset.y_test[ind].argmax(-1)
    pred = line_detector_model.predict_on_image(image).argmax(-1)

    fig = plt.figure(figsize=(15, 5))
    ax1 = fig.add_subplot(131)
    ax1.matshow(image, cmap='gray')
    ax2 = fig.add_subplot(132)
    ax2.matshow(gt, cmap='gray')
    ax3 = fig.add_subplot(133)
    ax3.matshow(pred, cmap='gray')

In [ ]:
# Let's also look at the training data augmentations

x_batch, y_batch = paragraphs_dataset.x_train[:4], paragraphs_dataset.y_train[:4]
x_batch_aug, y_batch_aug = line_detector_model.augment_batch(x_batch, y_batch)

for ind in range(x_batch_aug.shape[0]):
    fig = plt.figure(figsize=(15, 5))
    ax1 = fig.add_subplot(121)
    ax1.matshow(x_batch_aug[ind], cmap='gray')
    ax2 = fig.add_subplot(122)
    ax2.matshow(y_batch_aug[ind].argmax(-1), cmap='gray')